# Pipeline for Big Basket data --> need to run only Once
## If you have questions --> send to Roy Madpis

In [ ]:
! pip install clip
! pip install torch
! pip install torchvision
! pip install transformers


# Please change the following!:

In [ ]:
import os

### Original table location --> change to the path you have your catalog table stored
catalog_metadata_file_path = os.path.join("/content/drive/MyDrive/Capstone Project/Data/Big basket Data", "big_basket_combined.xlsx") ### big basket data frame

#### Images location --> change to the path you have your images stored in
catalog_images_location_folder = "/content/drive/MyDrive/Capstone Project/Data/Big basket Data_Images" ### big basket images directory

#### outputs locations:
### this is where we want to store the pre-processed table.
path_to_save_the_preprocessed_df = "/content/drive/MyDrive/Capstone Project/Data/Big basket Data/preprocessed.csv" #### where to save the new processed data frame ?

### this is where we want to save the embedding
loction_to_save_embeddings = "/content/drive/MyDrive/Capstone Project/Data/Big basket Data/embedding.csv" #### where to savethe description embedding?


#### This is the location where you want to save the category, sub-category and brand encoder-decoder
location_to_save_encoder = 'label_encoders.pkl' ### where to save the label encoder?? --> this is important for later --> when we want to get from the "encoded" category/brand the actual text-label

###### Change the following to matc your column names in your catalog data set (the one we read from `catalog_metadata_file_path`)
category_column_name = 'Category'  ### this is the name of the column in your catalog dataset that stores the Category
subcategory_column_name = 'Sub-Category'  ### this is the name of the column in your catalog dataset that stores the sub-category
sub_subcategory_column_name = 'Sub-sub-Category'  ### this is the name of the column in your catalog dataset that stores the sub-sub-category
brand_column_name = 'Brand' ### this is the name of the column in your catalog dataset that stores the brand
product_description_column_name = 'About the Product' ### this is the name of the column in your catalog dataset that stores the product's description
image_path_column_name = 'image_path' ### this is the name of the column in your catalog dataset that stores the product's description


###### cleaning data options:
## If there is a product **without description**, do you want to **remove it?** If yes, the following should be True; If NOT, and you wish to still use that product, then pass False
remove_products_without_description = True # True = Remove all products without description


In [ ]:
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import os
import torch
import clip
import datetime
import cv2
import seaborn as sns
from io import BytesIO

from PIL import Image
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import models, transforms
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split

import xml.etree.ElementTree as ET

device = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

try:
  ### Connect to my google drive
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("Could not connect to drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reading the catalog data

In [ ]:
try:
  df_catalog = pd.read_excel(catalog_metadata_file_path)
except:
  df_catalog = pd.read_csv(catalog_metadata_file_path)

df_catalog["index"] = df_catalog.index

print(df_catalog.shape)
display(df_catalog.head())

(114026, 12)


,Category,Sub-Category,Sub-sub-Category,Product Link,EAN Code,Image Link,Brand,SKU Name,SKU Size,MRP,About the Product,index
0,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1221014/soulflowe...,1221014,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,"Soulflower Cold Pressed Castor Oil, 2 x 225 ml...",2 x 225 ml,₹700,"100% pure, natural, vegan and cold-pressed Sou...",0
1,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079159/soulflow...,40079159,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Tea Tree Essential Oil - For Oily A...,30 ml,₹850,"100% pure, natural and vegan Soulflower Tea Tr...",1
2,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40324133/thriveco...,40324133,https://www.bigbasket.com/media/uploads/p/l/40...,ThriveCo,ThriveCo Rosemary Essential Oil - Boosts Hair ...,NaN,₹449,Rosemary oil stimulates blood circulation to t...,2
3,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079161/soulflow...,8904134017519,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Peppermint Essential Oil for Hair G...,NaN,₹450,"100% pure, natural and vegan Soulflower Pepper...",3
4,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1220833/soulflowe...,1220833,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,Soulflower Cold Pressed Olive Oil - For Dry Sk...,2x120 ml,₹430,"Soul flower high-end olive oil will soothe, co...",4


### Adding a column with the image location

In [ ]:
# df_catalog['image_path'] = np.nan

# # if 'image_path' not in df_catalog.columns:
# #     df_catalog['image_path'] = np.nan

# # Iterate through each row to download images and set the image path
# for idx, row in tqdm(df_catalog.iterrows(), total=df_catalog.shape[0], desc="Downloading images"):
#     image_url = row['Image Link']
#     image_index = row['index']
#     image_Brand = row['Brand']
#     image_extension = image_url.split('.')[-1]  # Get the image file extension (e.g., jpg, png)

#     # Define the image file path
#     image_file_path = os.path.join(catalog_images_location_folder, f"{image_Brand}_{image_index}.{image_extension}")

#     # Check if the image already exists
#     if os.path.exists(image_file_path):
#         #print(f"Image already exists: {image_file_path}, skipping download.")
#         # Set the image path in the dataframe
#         df_catalog.at[idx, 'image_path'] = image_file_path
#         continue  # Skip to the next image

# df_catalog.head()

  df_catalog.at[idx, 'image_path'] = image_file_path


,Category,Sub-Category,Sub-sub-Category,Product Link,EAN Code,Image Link,Brand,SKU Name,SKU Size,MRP,About the Product,index,image_path
0,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1221014/soulflowe...,1221014,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,"Soulflower Cold Pressed Castor Oil, 2 x 225 ml...",2 x 225 ml,₹700,"100% pure, natural, vegan and cold-pressed Sou...",0,/content/drive/MyDrive/Capstone Project/Data/B...
1,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079159/soulflow...,40079159,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Tea Tree Essential Oil - For Oily A...,30 ml,₹850,"100% pure, natural and vegan Soulflower Tea Tr...",1,/content/drive/MyDrive/Capstone Project/Data/B...
2,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40324133/thriveco...,40324133,https://www.bigbasket.com/media/uploads/p/l/40...,ThriveCo,ThriveCo Rosemary Essential Oil - Boosts Hair ...,NaN,₹449,Rosemary oil stimulates blood circulation to t...,2,/content/drive/MyDrive/Capstone Project/Data/B...
3,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079161/soulflow...,8904134017519,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Peppermint Essential Oil for Hair G...,NaN,₹450,"100% pure, natural and vegan Soulflower Pepper...",3,/content/drive/MyDrive/Capstone Project/Data/B...
4,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1220833/soulflowe...,1220833,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,Soulflower Cold Pressed Olive Oil - For Dry Sk...,2x120 ml,₹430,"Soul flower high-end olive oil will soothe, co...",4,/content/drive/MyDrive/Capstone Project/Data/B...


## Preprocessing

+ Changing column names so they align with our preprocessing / modeling configuration

In [ ]:
###rename some columns
df_catalog.rename(columns={
    category_column_name : 'masterCategory',
    subcategory_column_name: 'subCategory',
    sub_subcategory_column_name: 'Sub_sub_Category',
    brand_column_name:"brand",
    product_description_column_name:"Description",
    image_path_column_name:"image_path"}, inplace=True)


In [ ]:
df_catalog.head()

,masterCategory,subCategory,Sub_sub_Category,Product Link,EAN Code,Image Link,brand,SKU Name,SKU Size,MRP,Description,index,image_path
0,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1221014/soulflowe...,1221014,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,"Soulflower Cold Pressed Castor Oil, 2 x 225 ml...",2 x 225 ml,₹700,"100% pure, natural, vegan and cold-pressed Sou...",0,/content/drive/MyDrive/Capstone Project/Data/B...
1,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079159/soulflow...,40079159,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Tea Tree Essential Oil - For Oily A...,30 ml,₹850,"100% pure, natural and vegan Soulflower Tea Tr...",1,/content/drive/MyDrive/Capstone Project/Data/B...
2,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40324133/thriveco...,40324133,https://www.bigbasket.com/media/uploads/p/l/40...,ThriveCo,ThriveCo Rosemary Essential Oil - Boosts Hair ...,NaN,₹449,Rosemary oil stimulates blood circulation to t...,2,/content/drive/MyDrive/Capstone Project/Data/B...
3,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079161/soulflow...,8904134017519,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Peppermint Essential Oil for Hair G...,NaN,₹450,"100% pure, natural and vegan Soulflower Pepper...",3,/content/drive/MyDrive/Capstone Project/Data/B...
4,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1220833/soulflowe...,1220833,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,Soulflower Cold Pressed Olive Oil - For Dry Sk...,2x120 ml,₹430,"Soul flower high-end olive oil will soothe, co...",4,/content/drive/MyDrive/Capstone Project/Data/B...


### Remove rows with NA value in column 'image_path'
If there is a product without image path, then we cannot use that product for training/testing our model, so we remove these rows upfront


In [ ]:
### remove rows with NA value in column 'image_path'
num_rows_before = df_catalog.shape[0]
df_catalog.dropna(subset=['image_path'], inplace=True)

num_rows_after = df_catalog.shape[0]
rows_removed = num_rows_before - num_rows_after

print(f'Number of rows before: {num_rows_before}')
print(f'Number of rows removed: {rows_removed}')

print(f'Number of rows now: {df_catalog.shape[0]}')
df_catalog = df_catalog.reset_index(drop = True)

# Testing purposes:
# df_catalog = df_catalog.head(100)
df_catalog.shape

Number of rows before: 114026
Number of rows removed: 621
Number of rows now: 113405


(113405, 13)

## No more preprocessing from here!

Let's save the pre-processed df as pkl file, so we can read it in the next notebook which will train the Embedding model

In [ ]:
df = df_catalog.copy()
df.to_csv(path_to_save_the_preprocessed_df) ### Save the dataset


## We have a text column: "Description" --> tokenize --> embedding

In order to use the description of the product in the model, we need to tokenize the description column and then get the text embedding

I want you to write code that does that using a pretrained LLM

In [ ]:
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# Load the pretrained model and tokenizer (you can choose a different model if needed)
model_name = "distilbert-base-uncased"  # Or use "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

### Note: the following chunk will take a longgggggg time to run!, DONT GIVE UP!

In [ ]:
df_catalog.head()

,masterCategory,subCategory,Sub_sub_Category,Product Link,EAN Code,Image Link,brand,SKU Name,SKU Size,MRP,Description,index,image_path
0,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1221014/soulflowe...,1221014,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,"Soulflower Cold Pressed Castor Oil, 2 x 225 ml...",2 x 225 ml,₹700,"100% pure, natural, vegan and cold-pressed Sou...",0,/content/drive/MyDrive/Capstone Project/Data/B...
1,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079159/soulflow...,40079159,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Tea Tree Essential Oil - For Oily A...,30 ml,₹850,"100% pure, natural and vegan Soulflower Tea Tr...",1,/content/drive/MyDrive/Capstone Project/Data/B...
2,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40324133/thriveco...,40324133,https://www.bigbasket.com/media/uploads/p/l/40...,ThriveCo,ThriveCo Rosemary Essential Oil - Boosts Hair ...,NaN,₹449,Rosemary oil stimulates blood circulation to t...,2,/content/drive/MyDrive/Capstone Project/Data/B...
3,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/40079161/soulflow...,8904134017519,https://www.bigbasket.com/media/uploads/p/l/40...,Soulflower,Soulflower Peppermint Essential Oil for Hair G...,NaN,₹450,"100% pure, natural and vegan Soulflower Pepper...",3,/content/drive/MyDrive/Capstone Project/Data/B...
4,beauty & hygiene,skin care,aromatherapy,https://www.bigbasket.com/pd/1220833/soulflowe...,1220833,https://www.bigbasket.com/media/uploads/p/l/12...,Soulflower,Soulflower Cold Pressed Olive Oil - For Dry Sk...,2x120 ml,₹430,"Soul flower high-end olive oil will soothe, co...",4,/content/drive/MyDrive/Capstone Project/Data/B...


In [ ]:
# Set model to evaluation mode
model.eval()

# Tokenization and embedding generation function
def get_embeddings(text, tokenizer, model):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Generate embeddings using the model
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(**inputs)

    # Extract the embeddings (usually we take the last hidden state, mean over the sequence length)
    # outputs['last_hidden_state'] has shape (batch_size, sequence_length, hidden_size)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling over the sequence length

    return embeddings.squeeze().numpy()

In [ ]:
# Create a new DataFrame with 'index', 'image_path', and 'Description' columns
new_df = df_catalog.reset_index()[['image_path', 'Description']]

if remove_products_without_description:
  print("You chose to remove images without description! - Nice")
  ### remove rows with NA value in column 'image_path'
  num_rows_before = new_df.shape[0]
  new_df.dropna(subset=['Description'], inplace=True)

  num_rows_after = new_df.shape[0]
  rows_removed = num_rows_before - num_rows_after
  print(f'Number of rows before: {num_rows_before}')
  print(f'Number of rows removed: {rows_removed}')
  print(f'Number of rows now: {new_df.shape[0]}')
  new_df = new_df.reset_index(drop = True)

else:
  print("You chose to keep images without description! - great!, I am replacing NAs with empty string")
  new_df['Description'] = new_df['Description'].fillna('')  # Fill NaN with empty strings

# Apply the embedding function to the 'Description' column with a progress bar
tqdm.pandas()  # Initialize tqdm for pandas
new_df['embedding'] = new_df['Description'].progress_apply(lambda x: get_embeddings(x, tokenizer, model))


You chose to remove images without description! - Nice
Number of rows before: 113405
Number of rows removed: 4344
Number of rows now: 109061


  9%|▉         | 9992/109061 [27:37<4:18:05,  6.40it/s]

In [ ]:
# If you want to break down embeddings into individual columns, you can use pd.DataFrame
embedding_df = pd.DataFrame(new_df['embedding'].tolist())
# Concatenate the original DataFrame with the embeddings
final_df = pd.concat([df_catalog[['image_path', 'Description']], embedding_df], axis=1)

# Display the new DataFrame structure
print("final_df.shape:", final_df.shape)
final_df.head()

final_df.shape: (113405, 770)


,image_path,Description,0,1,2,3,4,5,6,7,...,758,759,760,761,762,763,764,765,766,767
0,/content/drive/MyDrive/Capstone Project/Data/B...,"100% pure, natural, vegan and cold-pressed Sou...",-0.220685,0.189505,0.265483,0.056628,0.460937,-0.033451,0.135180,0.452320,...,-0.094454,-0.307881,0.188101,-0.000182,0.125908,-0.203506,-0.141081,-0.071022,-0.279331,0.060411
1,/content/drive/MyDrive/Capstone Project/Data/B...,"100% pure, natural and vegan Soulflower Tea Tr...",-0.282873,0.159856,0.284020,0.041140,0.341671,0.013106,0.201740,0.398356,...,-0.203132,-0.285398,0.234261,0.011924,0.016644,-0.265068,-0.144999,-0.009189,-0.160574,0.018892
2,/content/drive/MyDrive/Capstone Project/Data/B...,Rosemary oil stimulates blood circulation to t...,0.023976,0.084641,0.333245,0.115848,0.410506,-0.203077,0.114444,0.254174,...,-0.073997,-0.185825,0.002539,-0.054753,0.159628,-0.204156,-0.031812,-0.189324,-0.185347,0.157606
3,/content/drive/MyDrive/Capstone Project/Data/B...,"100% pure, natural and vegan Soulflower Pepper...",-0.284317,0.134335,0.308609,0.099600,0.329050,-0.027204,0.228773,0.356667,...,-0.134838,-0.276468,0.216083,-0.009151,0.011475,-0.251627,-0.185996,-0.055083,-0.112323,0.076400
4,/content/drive/MyDrive/Capstone Project/Data/B...,"Soul flower high-end olive oil will soothe, co...",-0.066591,0.201716,0.369411,0.192352,0.346392,-0.039932,0.191706,0.342479,...,-0.197680,-0.160472,0.175327,-0.052500,0.077738,-0.187022,-0.086847,-0.214702,-0.113089,-0.137088


## Save the Embedding df as csv!

In [ ]:
### Step 1: Save locally
#df_embedding_file_name = "embedding.csv"

columns_to_save = ["image_path", "Description"] + list(final_df.columns[2:])
final_df[columns_to_save].to_csv(loction_to_save_embeddings, index=False)

print("Saved the preprocessed df in", loction_to_save_embeddings)

Saved the preprocessed df in /content/drive/MyDrive/Capstone Project/Data/Big basket Data/embedding.csv


## No more preprocessing from here!

Let's save the pre-processed df as pkl file, so we can read it in the next notebook which will train the Embedding model

In [ ]:
# df = df_catalog.copy()
# df.to_csv(path_to_save_the_df_with_desc_embedding) ### Save the dataset with the embedding ! --> this is super important since this is going to be the input for the embedding model!


'/content/drive/MyDrive/Capstone Project/Data/Big basket Data/preprocessed.csv'

# roymadpis